In [3]:
%load_ext autoreload
%autoreload 2

import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import torch
import torch.nn as nn
from evolver import CrossoverType, MutationType, MatrixEvolver
from models.unet import UNet
from dataset_utils import PartitionType
from landcover_dataloader import LandCoverDataset, create_land_cover_dataset_from_config, get_land_cover_dataloader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)  

In [13]:
params = {'batch_size': 8,
          'shuffle': False,
          'num_workers': 6}

dataset_dir = os.path.join(os.getcwd(), "data/landcover_small")
create_land_cover_dataset_from_config(dataset_dir)
train_loader = get_land_cover_dataloader(dataset_dir, PartitionType.TRAIN, params)
validation_loader = get_land_cover_dataloader(dataset_dir, PartitionType.VALIDATION, params)
finetuning_loader = get_land_cover_dataloader(dataset_dir, PartitionType.FINETUNING, params)
test_loader = get_land_cover_dataloader(dataset_dir, PartitionType.TEST, params)

In [14]:
import torch
torch.cuda.empty_cache()

import gc
del batch_x, batch_y, net
gc.collect()

NameError: name 'batch_x' is not defined

In [31]:
# Define Model Loss and Optimizers
max_epochs = 10
report_batches = 25
net = UNet(in_channels = 4, n_classes = 4, depth = 4)
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(max_epochs):
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        batch_x, batch_y = data
        batch_x, batch_y = batch_x.to(device), batch_y.to(device) 
        
        optimizer.zero_grad()
        outputs = net(batch_x)

        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if i % report_batches == report_batches - 1:
            print('TRAIN: [Epoch: %d,  Batch: %d, Epoch Percent: %0.2f, Avg Loss: %3d]' % (epoch + 1, 
                                                                                           i + 1, 
                                                                                           100 * train_loader.batch_size * (i + 1) / train_set.n_samples,
                                                                                        running_loss / (report_batches * train_loader.batch_size)))
        break
#             running_loss = 0.0
    
#     running_loss = 0.0
#     for i, data in enumerate(test_loader):
#         batch_x, batch_y = data
#         batch_x, batch_y = batch_x.to(device), batch_y.to(device) 
#         outputs = net(batch_x)
#         loss = criterion(outputs, batch_y)
#         running_loss += loss.item()

#     print('TEST: [Avg Batch Loss: %3d]' % (running_loss / ((i + 1) * test_loader.batch_size)))

In [ ]:
dropout_mask_evolver = MatrixEvolver([[256, 3]], CrossoverType.UNIFORM, MutationType.FLIP_BIT)

In [35]:
masks = {}

for epoch in range(max_epochs):
    running_loss = 0.0

    
    for i, data in enumerate(train_loader):
        batch_x, batch_y = data
        batch_x, batch_y = batch_x.to(device), batch_y.to(device) 
        masks["start"] = torch.randint(low=0, high=2, size=(256, 256)).to(device)
        net.set_dropout_masks(masks)        
        optimizer.zero_grad()
        outputs = net(batch_x)

        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if i % report_batches == report_batches - 1:
            print('TRAIN: [Epoch: %d,  Batch: %d, Epoch Percent: %0.2f, Avg Loss: %3d]' % (epoch + 1, 
                                                                                           i + 1, 
                                                                                           100 * train_loader.batch_size * (i + 1) / train_set.n_samples,
                                                                                        running_loss / (report_batches * train_loader.batch_size)))

tensor([[0, 0, 0,  ..., 1, 0, 1],
        [0, 0, 1,  ..., 1, 1, 1],
        [1, 0, 1,  ..., 1, 0, 1],
        ...,
        [0, 0, 0,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 1],
        [1, 1, 1,  ..., 1, 0, 0]], device='cuda:0')
2.0100601153232733
tensor([[0, 1, 0,  ..., 0, 1, 1],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 1, 1],
        ...,
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 1],
        [0, 1, 0,  ..., 0, 1, 0]], device='cuda:0')
2.001527043948325
tensor([[0, 0, 1,  ..., 1, 0, 0],
        [0, 1, 1,  ..., 1, 0, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 1, 1,  ..., 0, 1, 1],
        [0, 1, 0,  ..., 1, 1, 1],
        [0, 1, 0,  ..., 1, 0, 0]], device='cuda:0')
2.0056310441914555
tensor([[1, 0, 0,  ..., 1, 0, 0],
        [1, 1, 0,  ..., 0, 1, 0],
        [1, 1, 1,  ..., 0, 1, 0],
        ...,
        [0, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 1, 0],
        [1, 0, 0,  ..., 0, 0, 0]], device='cuda:0')
2.00

NameError: name 'train_set' is not defined

In [ ]:
# train initial model

# setup tensor board?
# model checkpoint saving
# accuracy, f1, loss

# finetune using backprop or using dropout
# if drop out at each epoch on finetuning generate new matrix
# save matrix sparsity or dropout percentage for all children
